In [1]:
#!pip install mediapipe opencv-python==4.5.5.64
#!pip install pyautogui
# source idea from https://www.youtube.com/watch?v=vQZ4IvB07ec&t=347s
# source https://google.github.io/mediapipe/solutions/hands
# полезно по оформлению https://habr.com/ru/post/485318/
# мышь https://myrusakov.ru/python-gui-automation.html

In [2]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import time
import math
import keyboard
import pyautogui
# from kalman_filtr import KalmanFilter   
from filterpy.kalman import KalmanFilter

In [3]:
pyautogui.size()  # Size(width=3840, height=2160)
pyautogui.PAUSE = 1
pyautogui.FAILSAFE = True # security

# Hands 
<img src=https://i.imgur.com/qpRACer.png />

In [4]:
# Вспомогательные фишечки
green = (0, 255, 0)
red = (0, 0, 255)
white = (255, 255, 255)
# cv2.putText(image, text, org, font, fontScale, color[,
#               thickness[, lineType[, bottomLeftOrigin]]])


def text(image, text, pos=(10, 30), fontScale=1, color=red, thickness=2):
    cv2.putText(image, text, pos, cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=fontScale, color=color, thickness=thickness)


def dist_betw_points(pt1, pt2):
    """ calc distance betw 2 points"""
    return math.sqrt((pt2[0]-pt1[0])*(pt2[0]-pt1[0]) +
                     (pt2[1]-pt1[1])*(pt2[1]-pt1[1]) + (pt2[2]-pt1[2])*(pt2[2]-pt1[2]))


def finger_connect_evt(pt1_tip, pt1_pip, pt2_tip, pt2_prev):
    """ connect finger event calc as a ratio of 2 distances - 
        first is the distance beetween finger ends, second - is 
        between pre-end finger points
    """
    # TODO!!
    pass


def pos_perst(hand, perst, coord):  # нужно ли это??? хз
    return hand.landmark[perst].coord


def move_pt(pt, pt_step, mult=1):
    return ((pt[0]-pt_step[0])*mult, (pt[1]-pt_step[1])*mult)


In [5]:
# TODO!
# сделать фильтр кальмана для сглаживания трека от которого будет мышка двигаться
# делать что-то для корректной работы, если есть вторая рука в кадре
#       например, работать с двумя руками, и брать в работу только то,

# DONE делать привязку к мышке
# DONE в первом приближении, возможно требует улучшений после привязки мыши.
#       DONE при фронтальном ракурсе руки соотношение длин, по которому принимается решение для
#       передвижения курсора оказывается ниже порога.

#       делать масштаб кисти для вычисления порога в зависимости от удаления от камеры.
#       тупое измерение расстояния между кончиками пальцев часто не дает эффекта, т.к.
#       эти кончики оказываются в разных местах в зависимости от ракурса.
#       поэтомуесть идея оценить взимное расоложение пальцев с использованием других точек
#       пальца.
# DONE разбираться как влияет z координата пока вроде без нее работает лучше чем с ней.
# DONE установить лимит на одну руку


#### Malfunc with front view 
<img src="pics/front_palm_front_malfunc.png" />

In [11]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cam = 2  # 0,1 - какая первая подключилась
cap = cv2.VideoCapture(cam)
while not cap.isOpened(): # await camera 
    pass
ret, frame = cap.read()
image_hight, image_width = frame.shape[:2]
index_tip = mp_hands.HandLandmark.INDEX_FINGER_TIP      # указательный конец
# index_dip = mp_hands.HandLandmark.INDEX_FINGER_DIP   # указательный 2 точка
index_pip = mp_hands.HandLandmark.INDEX_FINGER_PIP     # указательный 2 точка
thumb_tip = mp_hands.HandLandmark.THUMB_TIP             # большой конец
# thumb_ip = mp_hands.HandLandmark.THUMB_IP           # большой вторая точка
thumb_mcp = mp_hands.HandLandmark.THUMB_MCP            # большой вторая точка

dist_ratio_thresh = 3.2
dist_ends, dist_prevs = 1, 1
dist_ratio = 1
pt1_tip = (100, 100)
pt1_pip = (100, 100)
pt2_tip = (100, 100)
pt2_mcp = (100, 100)
pt1_mouse_pos_prev_cicle = (100, 100)   # предыдущая позиция курсора для отслеживания относительного перемещения
dp = (0, 0)             # дискрет перемещения
contact = False         # признак того что пальцы соединились

# секция инициализации фильтра калмана 
# отфильтруем для плавного перемещения курсора мыши !!!!! возможно отправить это в move
# kf.predict()
# z = np.array([[dp[0]], [dp[1]]])
# x_estimate = kf.x
# kf.update(z)
# print(f" {x_estimate}")
dt = 1.0/30  # time step
kf = KalmanFilter(dim_x=4, dim_z=2)
kf.x = np.array([0, 0, 0, 0])  # initial state (x, y, vx, vy)
kf.F = np.array([[1, 0, dt, 0],
                [0, 1, 0, dt],
                [0, 0, 1, 0],
                [0, 0, 0 ,1]])  # state transition matrix

kf.H = np.array([[1, 0, 0, 0], [0, 1, 0, 0]]) # measurement function
kf.P *= 1000.0 # initial state covariance
kf.R = np.array([[0.1, 0],[0, 0.1]]) # measurement noise covariance
kf.Q = np.array([[0.01, 0, 0.1, 0], [0, 0.01, 0, 0.1], [0.1, 0, 1, 0], [0, 0.1, 0, 1]]) # process noise covariance



with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.8, max_num_hands=2) as hands:
    while cap.isOpened():
        t0 = time.time()
        ret, frame = cap.read()
        if not ret: break
        
        # Detection
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # print(f"res {results}")

        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                # print(f" {mp_hands.HAND_CONNECTIONS}")
                mp_drawing.draw_landmarks(
                    image, hand, mp_hands.HAND_CONNECTIONS)

                pt1_tip = (hand.landmark[index_tip].x,
                           hand.landmark[index_tip].y, 0)
                pt1_pip = (hand.landmark[index_pip].x,
                            hand.landmark[index_pip].y, 0)

                pt2_tip = (hand.landmark[thumb_tip].x,
                           hand.landmark[thumb_tip].y, 0)
                pt2_mcp = (hand.landmark[thumb_mcp].x,
                            hand.landmark[thumb_mcp].y, 0)

                dist_ends = dist_betw_points(pt1_tip, pt2_tip)
                dist_ends = dist_ends if dist_ends != 0 else 0.01  # prevent div by zero

                dist_prevs = dist_betw_points(pt1_pip, pt2_mcp)

        contact = True if dist_ratio > dist_ratio_thresh else False

        # вычислить соотношение дистанций соответствующих точек пальцев
        dist_ratio = dist_prevs/dist_ends
        # вычислить перемещение мыши
        # сразу умножим движ на mult
        dp = move_pt(pt1_tip, pt1_mouse_pos_prev_cicle, mult=10000)
        
        
        if contact:
            # pyautogui.moveRel(-dp[0], -dp[1], _pause=False)
            # z = np.array([x, y]) # measurement (x, y)
            z = np.array([dp[0], dp[1]]) # measurement (x, y)
            kf.predict()
            kf.update(z)
            x, y, _, _ = kf.x
            pyautogui.moveRel(-x, -y, _pause=False)
        
        # переписать значение мыши предыдущего цикла
        # привязать мышь к концу указательного пальца
        pt1_mouse_pos_prev_cicle = pt1_tip

        time_spend = time.time() - t0

        color = green if contact else False
        text(image, f'{dist_ends:.2f} {dist_prevs:.2f} {dist_ratio:.2f}', color=color)
        text(image, f'{pt1_tip[0]:.2f} {pt1_tip[1]:.2f} : {dp[0]:.0f} {dp[1]:.0f} ',
                    pos=(300, 30), color=white, thickness=1)
        text(image, f'ts {time_spend:.3}', pos=(10, 60), color=white) # time spend indicator 0.02 for 2 hands
        # assert dp[0]*100<10, dp[0]

        image = cv2.resize(image, (image.shape[1]*3, image.shape[0]*3))  # optional
        cv2.imshow('hands', image)
        if cv2.waitKey(10) == 27:
            break
cap.release()
cv2.destroyAllWindows()


In [7]:
results.multi_hand_landmarks
# mp_hands.HAND_CONNECTIONS??
# mp_drawing.DrawingSpec??


[landmark {
   x: 0.6505443453788757
   y: 0.5681498050689697
   z: 6.844715585430095e-07
 }
 landmark {
   x: 0.5472425222396851
   y: 0.5628497004508972
   z: -0.03518681973218918
 }
 landmark {
   x: 0.4488500952720642
   y: 0.6091769933700562
   z: -0.0724320188164711
 }
 landmark {
   x: 0.36916735768318176
   y: 0.6577322483062744
   z: -0.09919608384370804
 }
 landmark {
   x: 0.30418723821640015
   y: 0.6857454180717468
   z: -0.12907934188842773
 }
 landmark {
   x: 0.48303845524787903
   y: 0.6610027551651001
   z: -0.13826698064804077
 }
 landmark {
   x: 0.436076283454895
   y: 0.8292425870895386
   z: -0.18793651461601257
 }
 landmark {
   x: 0.4003341495990753
   y: 0.9348405599594116
   z: -0.20754069089889526
 }
 landmark {
   x: 0.37420302629470825
   y: 1.0154690742492676
   z: -0.21581341326236725
 }
 landmark {
   x: 0.547007143497467
   y: 0.7209290862083435
   z: -0.13796992599964142
 }
 landmark {
   x: 0.48636701703071594
   y: 0.9151349067687988
   z: -0.182967

In [8]:
duration = 0
for i in range(0): # 5
    # pyautogui.moveRel(100, 0, duration=duration)
    pyautogui.moveRel(100, 0)
    pyautogui.moveRel(0, 100, duration=duration)
    pyautogui.moveRel(-100, 0, duration=duration)
    pyautogui.moveRel(0, -100, duration=duration)


In [9]:
import keyboard
import time

while False:# True:
    if keyboard.is_pressed('ctrl'):
        time.sleep(0.1)
        print(f"ctrl pressed")
        keyboard.release("ctrl")
        # keyboard.send("ctrl")
        # keyboard.unhook_all()
        # break


In [10]:
import filterpy
filterpy.__version__

'1.4.5'